In [ ]:
from datasets import load_dataset
import tqdm
import pandas as pd
from transformers import pipeline

: 

### QA - Process Klej_dyk

In [96]:
klej_dyk = load_dataset("allegro/klej-dyk")
klej_dyk_df_train = klej_dyk['train'].to_pandas()
klej_dyk_df_test = klej_dyk['test'].to_pandas()
# combine train and test
klej_dyk_df = pd.concat([klej_dyk_df_train, klej_dyk_df_test], ignore_index=True)
klej_dyk_df = klej_dyk_df[klej_dyk_df["target"]==1][["question","answer"]]
klej_dyk_df["question"] = klej_dyk_df["question"].str.capitalize()
#### CREATE PROMPTS
klej_dyk_df["question"] = '### Użytkownik: ' + klej_dyk_df["question"]
klej_dyk_df["answer"] = '### Asystent: ' + klej_dyk_df["answer"]
#### PROMPTS
prompts_klej_dyk = klej_dyk_df["question"] + klej_dyk_df["answer"]

### GPT-EXAM

In [80]:
gpt_exam = load_dataset("sdadas/gpt-exams")
gpt_exam_df = gpt_exam['train'].to_pandas()
#### CREATE PROMPTS
gpt_exam_df["question"] = '### Użytkownik: ' + gpt_exam_df["question"]
gpt_exam_df["answer"] = '### Asystent: ' + gpt_exam_df["answer"]
#### PROMPTS
prompts_gpt_exam = gpt_exam_df["question"] + gpt_exam_df["answer"]


### Guanco

In [4]:
oa_guanco = load_dataset("timdettmers/openassistant-guanaco")
oa_guanco_df = oa_guanco['train'].to_pandas()

Repo card metadata block was not found. Setting CardData to empty.


In [14]:
pipe = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection", truncation=True, max_length=128)
oa_guanco_en = []
for i,text in enumerate(tqdm.tqdm(oa_guanco_df["text"])):
    if pipe(text)[0]["label"] == "en":
        oa_guanco_en.append(text)

### temp df
df = pd.DataFrame(oa_guanco_en, columns=["text"])
df = df[~df["text"].str.contains("Open Assistant")]
df = df[~df["text"].str.contains("grammar")]
df = df[~df["text"].str.contains("Re-write this sentence")]
df = df[~df["text"].str.contains("Rewrite")]

## replace ### Asystent with <ASYSTENT_TAG>
df["text"] = df["text"].str.replace("### Assistant", "<ASYSTENT_TAG>")
## replace ### Użytkownik with <USER_TAG>
df["text"] = df["text"].str.replace("### Human", "<USER_TAG>")

df.to_csv("../data/en/oa_guanco_en.csv", index=False)

  1%|          | 74/9578 [00:05<11:54, 13.30it/s]


KeyboardInterrupt: 

In [17]:
## replace ### Asystent with <ASYSTENT_TAG>
df["text"] = df["text"].str.replace("### Assistant", "<ASYSTENT_TAG>")
## replace ### Użytkownik with <USER_TAG>
df["text"] = df["text"].str.replace("### Human", "<USER_TAG>")
df.to_csv("../data/en/oa_guanco_en.csv", index=False)

In [82]:
oa_translated_df = pd.read_csv("oa_dataset_translated.csv")
oa_translated_df = oa_translated_df["translation"]
## create new column in new df named oa_prompts with translation
oa_translated_df = oa_translated_df.str.replace("<ASYSTENT_TAG>", "### Asystent")
prompts_oa = oa_translated_df.str.replace("<USER_TAG>", "### Użytkownik")

#### Dolly 15K

In [ ]:
dolly = load_dataset("databricks/databricks-dolly-15k")
dolly_train_df = dolly['train'].to_pandas()
# to csv
dolly_train_df.to_csv("../data/en/dolly_train.csv", index=False)

: 

### QA - with context - Process clarin-pl/poquad

In [ ]:
poquad = load_dataset("clarin-pl/poquad")
poquad_df_train = poquad['train'].to_pandas()
poquad_df_test = poquad['validation'].to_pandas()
poquad_df = pd.concat([poquad_df_train, poquad_df_test], ignore_index=True)
poquad_df = poquad_df.drop_duplicates(subset=["title"]).reset_index(drop=True)
poquad_df["answers"] = poquad_df["answers"].apply(lambda x: x["text"][0])
poquad_df["answers"] = poquad_df["answers"].str.capitalize()
poquad_df = poquad_df[["context","question","answers"]]
poquad_df
#### CREATE PROMPTS
poquad_df["question_uzytownik"] = '### Użytkownik: ' + poquad_df["question"]
poquad_df["answer_asystent"] = '### Asystent: ' + poquad_df["answers"]
#### PROMPTS
"Na podstawie poniższego tekstu odpowiedz na pytanie: ",
"q" + "odpowiedz korzystając z tekstu" + "a"


### Process poleval_2019_mt

In [113]:
opus100 = load_dataset("opus100", "en-pl")

In [118]:
opus100_df = opus100['validation'].to_pandas()
#get data from dictonary inside column translation
opus100_df["en"] = opus100_df["translation"].apply(lambda x: x["en"])
opus100_df["pl"] = opus100_df["translation"].apply(lambda x: x["pl"])
## przetłumacz teksty z angielskiego na polski z pierwszego 1000 wierszy
# Take first 1000 rows
opus100_df_en_pl = opus100_df[:1000]
opus100_df_en_pl["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Angielskiego na Polski." + opus100_df_en_pl["en"] + "### Asystent: " + opus100_df_en_pl["pl"]

opus100_df_pl_en = opus100_df[1000:2000]
opus100_df_pl_en["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Polskiego na Angielski." + opus100_df["pl"] + "### Asystent: " + opus100_df["en"]

#### concat 
prompts_opus100 = pd.concat([opus100_df_en_pl["prompt"], opus100_df_pl_en["prompt"]], ignore_index=True)
# reset index
prompts_opus100 = prompts_opus100.reset_index(drop=True)


/var/folders/fn/clqd998j3vn03yyqqybzkq240000gn/T/ipykernel_73466/3943497898.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opus100_df_en_pl["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Angielskiego na Polski." + opus100_df_en_pl["en"] + "### Asystent: " + opus100_df_en_pl["pl"]
/var/folders/fn/clqd998j3vn03yyqqybzkq240000gn/T/ipykernel_73466/3943497898.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opus100_df_pl_en["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Polsk

### JOIN PROMPTS

In [129]:
## concat all prompts
prompts = pd.concat([prompts_klej_dyk, prompts_gpt_exam, prompts_oa, prompts_opus100], ignore_index=True)
## shuffle rows in prompts 
prompts = prompts.sample(frac=1).reset_index(drop=True)
## add column NAMED text
prompts = pd.DataFrame(prompts, columns=["text"])
prompts

,text
0,### Użytkownik: Co to jest alergia i jakie są ...
1,### Użytkownik: W jaki sposób atmosfera wpływa...
2,### Użytkownik: Skategoryzuj ten element wiado...
3,### Użytkownik: Jakie są główne organy władzy ...
4,### Użytkownik: Wygeneruj kreatywny pomysł na ...
...,...
19762,### Użytkownik: Jakie są podstawowe zasady erg...
19763,### Użytkownik: Jakie są główne zagrożenia zwi...
19764,### Użytkownik: Przetłumacz dany tekst z język...
19765,### Użytkownik: Jakie są główne społeczności ż...


### Post to huggingface dataset

In [134]:
from datasets import load_dataset
import datasets
from datasets import Dataset, DatasetDict

### get random 1000 rows from prompts with seed and put into validation df
prompts_sample = prompts.sample(n=500, random_state=1)
prompts_sample = prompts_sample.reset_index(drop=True)
### substract validation df from prompts df
prompts_train = prompts[~prompts.text.isin(prompts_sample.text)]
prompts_train = prompts_train.reset_index(drop=True)
### reset index

In [137]:
ds = DatasetDict()
ds['train'] = Dataset.from_pandas(prompts_train)
ds['validation'] = Dataset.from_pandas(prompts_sample)

In [139]:
ds.push_to_hub("szymonrucinski/krakowiak-pl")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.28s/it]
